In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs
th.backends.cudnn.benchmark = True

In [ ]:
sensor_names = ['Fz', 
                'FC3','FC1','FCz','FC2','FC4',
                'C5','C3','C1','Cz','C2','C4','C6',
                'CP3','CP1','CPz','CP2','CP4',
                'P1','Pz','P2',
                'POz']
orig_train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat')
train_cnt = orig_train_cnt.reorder_channels(sensor_names)

train_inputs = create_inputs(train_cnt, final_hz=256, half_before=True)
n_split = len(train_inputs[0]) - 40
test_inputs = [t[-40:] for t in train_inputs]
train_inputs = [t[:-40] for t in train_inputs]

In [ ]:
cuda = True
if cuda:
    train_inputs = [i.cuda() for i in train_inputs]
    test_inputs = [i.cuda() for i in test_inputs]

In [ ]:
from reversible2.graph import Node
from reversible2.branching import CatChans, ChunkChans, Select
def invert(feature_model, out):
    return feature_model.invert(out)

from copy import deepcopy
from reversible2.graph import Node
from reversible2.distribution import TwoClassDist
from reversible2.wrap_invertible import WrapInvertible
from reversible2.blocks import dense_add_const, conv_add_3x3_const
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter

set_random_seeds(2019011641, cuda)
n_chans = train_inputs[0].shape[1]
n_time = train_inputs[0].shape[2]
base_model = nn.Sequential(
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
                   grad_is_inverse=True, keep_input=True),# 2 x 256
    conv_add_3x3_const(2*n_chans,32),
    conv_add_3x3_const(2*n_chans,32),
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=True), grad_is_inverse=True), # 4 x 128
    conv_add_3x3_const(4*n_chans,32),
    conv_add_3x3_const(4*n_chans,32),
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=True), grad_is_inverse=True), # 8 x 64
    conv_add_3x3_const(8*n_chans,32),
    conv_add_3x3_const(8*n_chans,32, keep_output=True))
base_model.cuda();

branch_1_a =  nn.Sequential(
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=False), 
                   grad_is_inverse=True, keep_input=True), # 8 x 32
    conv_add_3x3_const(8*n_chans,32),
    conv_add_3x3_const(8*n_chans,32),
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
                   grad_is_inverse=True),# 16 x 16
    conv_add_3x3_const(16*n_chans,32),
    conv_add_3x3_const(16*n_chans,32),
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
                   grad_is_inverse=True), # 32 x 8
    conv_add_3x3_const(32*n_chans,32),
    conv_add_3x3_const(32*n_chans,32, keep_output=True),
)
branch_1_b = nn.Sequential(
    *(list(deepcopy(branch_1_a).children()) + [
    WrapInvertible(ViewAs((-1, 32*n_chans,n_time//64,1), (-1,(n_time // 2)*n_chans)),
                   grad_is_inverse=True, keep_input=True),
    dense_add_const((n_time // 2)*n_chans,32),
    dense_add_const((n_time // 2)*n_chans,32),
    dense_add_const((n_time // 2)*n_chans,32),
    dense_add_const((n_time // 2)*n_chans,32, keep_output=True),
]))
branch_1_a.cuda();
branch_1_b.cuda();

branch_2_a = nn.Sequential(
    WrapInvertible(SubsampleSplitter(stride=[2,1], chunk_chans_first=False),
                   keep_input=True, grad_is_inverse=True),# 32 x 4
    conv_add_3x3_const(32*n_chans,32),
    conv_add_3x3_const(32*n_chans,32),
    WrapInvertible(SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
                   grad_is_inverse=True),# 64 x 2
    conv_add_3x3_const(64*n_chans,32),
    conv_add_3x3_const(64*n_chans,32),
    WrapInvertible(ViewAs((-1, (n_time // 4)*n_chans,1,1), (-1,(n_time // 4)*n_chans)),
                   grad_is_inverse=True),
    dense_add_const((n_time // 4)*n_chans,64),
    dense_add_const((n_time // 4)*n_chans,64),
    dense_add_const((n_time // 4)*n_chans,64),
    dense_add_const((n_time // 4)*n_chans,64, keep_output=True),
)


branch_2_b = deepcopy(branch_2_a).cuda()
branch_2_a.cuda();
branch_2_b.cuda();

final_model = nn.Sequential(
    dense_add_const(n_time*n_chans,256,keep_input=True),
    dense_add_const(n_time*n_chans,256),
    dense_add_const(n_time*n_chans,256),
    dense_add_const(n_time*n_chans,256),
    WrapInvertible(RFFT(), keep_output=True),
)
final_model.cuda();
o = Node(None, base_model)
o = Node(o, ChunkChans(2))
o1a = Node(o, Select(0))
o1b = Node(o, Select(1))
o1a = Node(o1a, branch_1_a)
o1b = Node(o1b, branch_1_b)
o2 = Node(o1a, ChunkChans(2))
o2a = Node(o2, Select(0))
o2b = Node(o2, Select(1))
o2a = Node(o2a, branch_2_a)
o2b = Node(o2b, branch_2_b)
o = Node([o1b,o2a,o2b], CatChans())
o = Node(o, final_model)
feature_model = o
if cuda:
    feature_model.cuda()
feature_model.eval();

### set feature model weights and distribution to good start parameters

In [ ]:
from reversible2.constantmemory import clear_ctx_dicts
from reversible2.distribution import TwoClassDist

feature_model.data_init(th.cat((train_inputs[0], train_inputs[1]), dim=0))

# Check that forward + inverse is really identical
t_out = feature_model(train_inputs[0][:2])
inverted = invert(feature_model, t_out)
clear_ctx_dicts(feature_model)
assert th.allclose(train_inputs[0][:2], inverted, rtol=1e-3,atol=1e-4)
device = list(feature_model.parameters())[0].device
from reversible2.ot_exact import ot_euclidean_loss_for_samples
class_dist = TwoClassDist(2, np.prod(train_inputs[0].size()[1:]) - 2)
class_dist.cuda()

for i_class in range(2):
    with th.no_grad():
        this_outs = feature_model(train_inputs[i_class])
        mean = th.mean(this_outs, dim=0)
        std = th.std(this_outs, dim=0)
        class_dist.set_mean_std(i_class, mean, std)
        # Just check
        setted_mean, setted_std = class_dist.get_mean_std(i_class)
        assert th.allclose(mean, setted_mean)
        assert th.allclose(std, setted_std)
clear_ctx_dicts(feature_model)

optim_model = th.optim.Adam(feature_model.parameters(), lr=1e-4, betas=(0,0.9))
optim_dist = th.optim.Adam(class_dist.parameters(), lr=1e-2, betas=(0,0.9))

In [ ]:

class ResidualBlock(nn.Module):
    def __init__(self, F):
        super(ResidualBlock, self).__init__()
        self.F = F

    def forward(self, x):
        return self.F(x) + x


from reversible2.discriminator import ProjectionDiscriminator
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,1), stride=1, padding=(1,0),bias=True)),
    )
    


from braindecode.torch_ext.util import set_random_seeds
adv_model = nn.Sequential(
    nn.Conv2d(22,64, (3,1), stride=1, padding=(1,0),bias=True), #256
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), #128
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), # 64
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), # 32
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), # 16
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), # 8
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), # 4
    res_block(64,96),
    res_block(64,96),
    nn.AvgPool2d((2,1)), # 2
    res_block(64,96),
    res_block(64,96),
    ViewAs((-1,64,2,1), (-1,128)),
    )

adv_model = ProjectionDiscriminator(adv_model,128,2,)
adv_model.cuda();

optim_adv = th.optim.Adam([{
    'params': adv_model.parameters(),
    'lr': 4e-4, 'weight_decay': 0.00}],#lr 0.0004
                         betas=(0,0.9))
from reversible2.graph import data_init_sequential_module
_ = data_init_sequential_module(adv_model.adv_feature_model, th.cat((train_inputs[0], train_inputs[1]), dim=0))

In [ ]:
import pandas as pd
df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
g_grad_norm = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
gradient_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan
d_grad_norm = np.nan
ot_out_loss_other = np_to_var([np.nan],dtype=np.float32)
ot_out_loss = np_to_var([np.nan],dtype=np.float32)


In [ ]:
from reversible2.constantmemory import clear_ctx_dicts
from reversible2.timer import Timer
from plot import plot_outs
from reversible2.gradient_penalty import gradient_penalty


i_start_epoch_out = 401
n_epochs = 5001
gen_frequency = 10
for i_epoch in range(n_epochs):
    epoch_row = {}
    with Timer(name='EpochLoop', verbose=False) as loop_time:
        gen_update = (i_epoch % gen_frequency) == (gen_frequency-1)
            
        optim_model.zero_grad()
        optim_dist.zero_grad()
        optim_adv.zero_grad()
        for i_class in range(len(train_inputs)):
            y = np_to_var([i_class]).cuda()
            class_ins = train_inputs[i_class]
            other_class_ins = train_inputs[1-i_class]
            with th.set_grad_enabled(gen_update):
                samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 4)
                inverted = feature_model.invert(samples)
            
            with th.set_grad_enabled(gen_update):
                outs = feature_model(class_ins)
                changed_to_other_class = class_dist.change_to_other_class(outs, i_class_from=i_class,
                                                                          i_class_to=1-i_class)
                other_inverted = feature_model.invert(changed_to_other_class)
            if not gen_update:
                score_fake = adv_model(inverted.detach(), y)
                score_real = adv_model(class_ins, y)
                gradient_loss = gradient_penalty(adv_model, class_ins, inverted[:(len(class_ins))].detach(), y)
                d_loss = -score_real.mean() + score_fake.mean() + gradient_loss * 100
                d_loss.backward()
                
                score_real_other = adv_model(other_class_ins.detach(), 1-y)
                score_fake_other = adv_model(other_inverted.detach(), 1-y)
                gradient_loss_other = gradient_penalty(adv_model, other_class_ins, 
                                                 other_inverted[:(len(class_ins))].detach(), 1-y)
                d_loss_other =  -score_real_other.mean() + score_fake_other.mean() + gradient_loss_other * 100
                d_loss_other.backward()
                
                # Clip gradient
                d_grad_norm = np.mean([th.nn.utils.clip_grad_norm_([p],100) for p in adv_model.parameters()])
                d_grad = np.mean([th.sum(p.grad **2).item() for p in adv_model.parameters()])
                wd_d = -(-score_real.mean() + score_fake.mean()).item()
                epoch_row['wd_d_{:d}'.format(i_class)] = wd_d
            else:
                ot_out_loss = ot_euclidean_loss_for_samples(samples[:,:2], outs[:,:2])
                other_samples = class_dist.get_samples(1-i_class, len(train_inputs[1-i_class]) * 4)
                ot_out_loss_other = ot_euclidean_loss_for_samples(other_samples[:,:2], changed_to_other_class[:,:2])
                score_fake = adv_model(inverted, y)
                score_fake_other = adv_model(other_inverted, 1-y)
                g_loss = -th.mean(score_fake) - th.mean(score_fake_other) + ot_out_loss + ot_out_loss_other
                g_loss.backward()
                # Clip gradient
                g_grad_norm = np.mean([th.nn.utils.clip_grad_norm_([p],100) for p in feature_model.parameters()])
                g_grad = np.mean([th.sum(p.grad **2).item() for p in feature_model.parameters()])
            clear_ctx_dicts(feature_model)
                
            """
            assert False
            ot_loss_in = ot_euclidean_loss_for_samples(class_ins.view(class_ins.shape[0], -1),
                                                       inverted.view(inverted.shape[0], -1))
            del inverted
            with th.set_grad_enabled(gen_update):
                outs = feature_model(class_ins)
            if i_epoch < i_start_epoch_out:
                ot_loss_out = th.zeros(1, device=class_ins.device)
            else:
                ot_loss_out = ot_euclidean_loss_for_samples(outs[:,:2].squeeze(), samples[:,:2].squeeze())
            del samples
                
            other_class_ins = train_inputs[1-i_class]
            changed_to_other_class = class_dist.change_to_other_class(outs, i_class_from=i_class, i_class_to=1-i_class)
            other_inverted = feature_model.invert(changed_to_other_class)
            ot_transformed_in = ot_euclidean_loss_for_samples(other_class_ins.view(other_class_ins.shape[0], -1),
                                                              other_inverted.view(other_inverted.shape[0], -1))
            
            if i_epoch < i_start_epoch_out:
                ot_transformed_out = th.zeros(1, device=class_ins.device)
            else:
                other_samples = class_dist.get_samples(1-i_class, len(train_inputs[i_class]) * 2)
                ot_transformed_out = ot_euclidean_loss_for_samples(changed_to_other_class[:,:2].squeeze(),
                                                                   other_samples[:,:2].squeeze(),)
            loss =  ot_loss_out + ot_transformed_in + ot_transformed_out + ot_loss_in
            loss.backward()
            del outs, other_class_ins, other_inverted
            clear_ctx_dicts(feature_model)"""
        if not gen_update:
            optim_adv.step()
        else:
            optim_model.step()
            optim_dist.step()
    epoch_row.update({
    'd_loss': d_loss.item(),
    'g_loss': g_loss.item(),
    'grad_loss': gradient_loss.item(),
    'o_real': th.mean(score_real).item(),
    'o_fake': th.mean(score_fake).item(),
    'g_grad': g_grad,
    'g_grad_norm': g_grad_norm,
    'd_grad': d_grad,
    'd_grad_norm': d_grad_norm,
        'ot_out_loss': ot_out_loss.item(),
        'ot_out_loss_other': ot_out_loss_other.item(),
    'runtime': loop_time.elapsed_secs * 1000})
    if i_epoch % (n_epochs // 20) != 0:
        df = df.append(epoch_row, ignore_index=True)
        # otherwise add ot loss in
    else:
        for i_class in range(len(train_inputs)):
            with th.no_grad():
                samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 4)
                inverted = feature_model.invert(samples)
                clear_ctx_dicts(feature_model)
                ot_loss_in = ot_euclidean_loss_for_samples(class_ins.view(class_ins.shape[0], -1),
                                                           inverted.view(inverted.shape[0], -1)[:(len(class_ins))])
                epoch_row['ot_loss_in_{:d}'.format(i_class)] = ot_loss_in.item()
        df = df.append(epoch_row, ignore_index=True)
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("G Loss: {:.2E}".format(g_loss.item()))
        print("D Loss: {:.2E}".format(d_loss.item()))
        print("OT Loss In: {:.2E}".format(ot_loss_in.item()))
        #print("OT Loss Out: {:.2E}".format(ot_loss_out.item()))
        #print("Transformed OT Loss In: {:.2E}".format(ot_transformed_in.item()))
        #print("Transformed OT Loss Out: {:.2E}".format(ot_transformed_out.item()))
        print("Loop Time: {:.0f} ms".format(loop_time.elapsed_secs * 1000))
        display(df.iloc[-3:])
        plot_outs(feature_model, train_inputs, test_inputs,
                 class_dist)
        fig = plt.figure(figsize=(8,2))
        plt.plot(var_to_np(th.cat((th.exp(class_dist.class_log_stds),
                                 th.exp(class_dist.non_class_log_stds)))),
                marker='o')
        display_close(fig)
    


In [ ]:
with th.no_grad():
    for n_samples_factor in [2,1]:
        for setname, set_inputs in (("Train", train_inputs), ("Test", test_inputs)):
            total_loss = 0
            total_spec_ot = 0
            for i_class in range(2):
                class_ins = set_inputs[i_class]
                samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * n_samples_factor)
                inverted = feature_model.invert(samples)
                loss = ot_euclidean_loss_for_samples(class_ins.view(class_ins.shape[0],-1),
                                                     inverted.view(inverted.shape[0],-1))
                total_loss += loss
                spec_ot = ot_euclidean_loss_for_samples(
                    th.rfft(class_ins.squeeze(), signal_ndim=1, normalized=True).view(class_ins.shape[0],-1),
                    th.rfft(inverted.squeeze(), signal_ndim=1, normalized=True).view(inverted.shape[0],-1))
                total_spec_ot += spec_ot
                clear_ctx_dicts(feature_model)
            print("{:d} Samples ". format(len(samples)) + setname + " Loss: {:.1f}".format(total_loss.item() / 2))
            print("{:d} Samples ". format(len(samples)) + setname + " SpecLoss: {:.1f}".format(spec_ot.item() / 2))

    # Show losses between sets
    for setname, set_inputs in (("Train", train_inputs),):
        total_loss = 0
        total_spec_ot = 0
        for i_class in range(2):
            class_ins = set_inputs[i_class]
            samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 5) # take same number of samples
            inverted = test_inputs[i_class]
            loss = ot_euclidean_loss_for_samples(class_ins.view(class_ins.shape[0],-1),
                                                     inverted.view(inverted.shape[0],-1))
            total_loss += loss
            spec_ot = ot_euclidean_loss_for_samples(
                th.rfft(class_ins.squeeze(), signal_ndim=1, normalized=True).view(class_ins.shape[0],-1),
                th.rfft(inverted.squeeze(), signal_ndim=1, normalized=True).view(inverted.shape[0],-1))
            total_spec_ot += spec_ot
        print(setname + " Loss: {:.1f}".format(total_loss.item() / 2))
        print(setname + " SpecLoss: {:.1f}".format(spec_ot.item() / 2))


In [ ]:
from reversible2.plot import plot_head_signals_tight

mean_bps_per_class = []
for i_class in range(len(train_inputs)):
    inverted = train_inputs[i_class]
    mean_bps_per_class.append(
        np.mean(np.abs(np.fft.rfft(var_to_np(inverted.squeeze()))), axis=0))

fig = plt.figure(figsize=(8,3))
plt.plot(np.mean(mean_bps_per_class, axis=1).T)
display_close(fig)

fig = plot_head_signals_tight(np.stack(mean_bps_per_class, axis=-1), sensor_names=sensor_names,
                             figsize=(20,12));

In [ ]:
from reversible2.plot import plot_head_signals_tight

mean_bps_per_class = []
for i_class in range(len(train_inputs)):
    samples = class_dist.get_samples(i_class, 400)
    inverted = feature_model.invert(samples)
    mean_bps_per_class.append(
        np.mean(np.abs(np.fft.rfft(var_to_np(inverted.squeeze()))), axis=0))

fig = plt.figure(figsize=(8,3))
plt.plot(np.mean(mean_bps_per_class, axis=1).T)
display_close(fig)

fig = plot_head_signals_tight(np.stack(mean_bps_per_class, axis=-1), sensor_names=sensor_names,
                             figsize=(20,12));

In [ ]:
fig = plot_head_signals_tight(np.log(mean_bps_per_class[0]/mean_bps_per_class[1]), sensor_names=sensor_names,
                             figsize=(20,12));

In [ ]:
for i_class in range(2):
    samples = class_dist.get_samples(i_class, 20)
    inverted = feature_model.invert(samples)
    plot_head_signals_tight(var_to_np(inverted)[0].squeeze(), sensor_names=sensor_names,
                             figsize=(20,12));

In [ ]:
np.array(df.ot_loss_in)[~np.isnan(df.ot_loss_in)]

In [ ]:
plt.plot(df.ot_loss_in)

In [ ]:
len(df)

In [ ]:
df.O